In [1]:
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt

##### 注意
在补充习题中不得使用任何for循环。

# Ex1：NumPy的向量化运算
### 1、
给定一个正整数列表，请找出缺失的最小正整数。

In [2]:
def find_miss(arr):
    arr[arr[arr <= len(arr)] - 1] = 0
    ans = np.nonzero(arr)
    return (ans[0][0] if len(ans[0]) else len(arr)) + 1

In [4]:
arr = np.array([6,3,5,1,2])
find_miss(arr)

4

In [30]:
arr = np.array([6,3,5,1,2])
arr

array([6, 3, 5, 1, 2])

In [31]:
arr <= len(arr)

array([False,  True,  True,  True,  True])

In [33]:
arr[arr <= len(arr)] - 1

array([2, 4, 0, 1])

In [27]:
arr[arr[arr <= len(arr)] - 1] = 0
arr

array([0, 0, 0, 1, 0])

In [28]:
ans = np.nonzero(arr)
ans

(array([3], dtype=int64),)

In [29]:
(ans[0][0] if len(ans[0]) else len(arr)) + 1

4

### 2、
设计一个生成二维NumPy数组的函数get_res()，其输入为正整数n，返回的数组构造方式如下：第1行填入1个1，第2行在上一行填入位置的下一列连续填入2个2，第3行在第二行最后一个填入位置的下一列连续填入3个3，…，第n行在第n-1行最后一个填入位置的下一列连续填入n个n。

In [5]:
def get_res(n):
    ans = np.zeros((n, n * (n+1) // 2))
    x_idx = np.repeat(range(1, n+1), range(1, n+1))
    ans[x_idx-1, range(ans.shape[1])] = x_idx
    return ans

In [6]:
n = 5
get_res(n)

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 2., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 3., 3., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 4., 4., 4., 4., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 5., 5., 5., 5., 5.]])

### 3、
点 $A$ 初始位置在数轴原点处，现进行n步简单随机游走，即每一步以等概率向左或向右移动长度为1的距离，记该点最终的位置为 $S_n$ ，则可以证明
$$\lim_{n\rightarrow+\infty}\frac{\mathbb{E}|S_n|}{\sqrt{n}}=\sqrt{\frac{2}{\pi}}$$
 
 
 
现取n为5000进行1000次试验，且每次试验中用 $\frac{1}{100}\sum_{k=1}^{100}|S_n|$
 
 来代替 $\mathbb{E}|S_n|$ 。此时可以计算得到1000个 $\mathbb{E}|S_n|$
 
 
 的估计值，请计算这些估计值的均值、0.05分位数和0.95分位数。

In [7]:
walks = np.array([-1, 1])[np.random.randint(0, 2, (1000, 100, 5000))]
estimate = np.abs(walks.sum(-1)).mean(-1) / np.sqrt(5000) - np.sqrt(2 / np.pi)

In [8]:
estimate.mean()

0.0011766489508805264

In [9]:
np.quantile(estimate, 0.05)

-0.09793370024192573

In [10]:
np.quantile(estimate, 0.95)

0.09987235072719895

### 4、
在二维平面上有n个点，每个点具有k维特征，点的坐标数据记录在node_xy中，点的特征数据记录在node_fea中。现要计算所有点的相关矩阵 $S$ ，点a和点b的相关系数定义如下
$$S_{ab} = \frac{\sigma_{ab}}{2} + \frac{\lambda_{ab}}{2}$$
 
 
其中，若记点a特征为 $A$ ，点b特征为 $B$ ，则有
$$\sigma_{ab} = \frac{\sum_{i=1}^kA_iB_i}{\sqrt{\sum_{i=1}^kA^2_i}\sqrt{\sum_{i=1}^kB^2_i}}$$
 
对于点a而言，将所有点到点a的二维平面距离进行排序，从而得到每个点到点a的距离排名，距离最近（排名为1）的点是点a自身，记点b的排名为 $r^{(a)}_b$
 ，则定义
$$\lambda_{ab} = 1 - \frac{2\times (r^{(a)}_b-1)}{n-1}$$
 
请对于给定的node_xy和node_fea计算相关矩阵 $S$ 。（提示：使用np.argsort()）

In [11]:
def get_S(node_xy, node_fea):
    sq_sum = (node_fea ** 2).sum(-1)
    sigma = (node_fea @ node_fea.T) / np.sqrt(sq_sum[:, None] * sq_sum)
    x, y = node_xy[:, 0], node_xy[:, 1]
    dist = np.sqrt((x - x[:, None]) ** 2 + (y - y[:, None]) ** 2)
    rank = dist.argsort(1).argsort(1)
    lamda = 1 - 2 * rank / (node_fea.shape[0] - 1)
    return (sigma + lamda) / 2

In [12]:
n, k = 5, 2
np.random.seed(1)
node_xy = np.random.rand(n, 2)
node_fea = np.random.rand(n, k)
get_S(node_xy, node_fea)

array([[ 1.        ,  0.22457492, -0.0631893 ,  0.49786201,  0.74886174],
       [-0.02542508,  1.        ,  0.49119664,  0.70800423,  0.21287902],
       [-0.0631893 ,  0.74119664,  1.        ,  0.41246708,  0.16940849],
       [-0.00213799,  0.70800423,  0.41246708,  1.        ,  0.2498435 ],
       [ 0.74886174,  0.21287902, -0.08059151,  0.4998435 ,  1.        ]])

# Ex2：统计学生的成绩情况
在data/supplement/ex2目录下存放了某校高三第一学期的学生成绩情况，包含16次周测成绩、期中考试成绩和期末考试成绩，科目一栏的成绩表示学生选课的成绩。所有的表中，相同的行表示的是同一位同学。请完成以下练习：

In [227]:
df = pd.read_csv('data/supplement/ex2/第1次周测成绩.csv')

df.head()

,班级,姓名,选科,语文,数学,英语,科目
0,1,吴刚,地理,93,95,82,69
1,1,卢楠,物理,108,77,90,94
2,1,唐秀兰,历史,88,72,95,85
3,1,张刚,化学,85,88,102,76
4,1,姜洋,历史,104,99,84,86


### 1、
该校高三年级中是否存在姓名相同的学生？

In [13]:
df = pd.read_csv("data/supplement/ex2/第1次周测成绩.csv")
df.姓名.nunique() == df.shape[0]

True

### 2、
在第一次周测中，请求出每个班级选修物理或化学同学的语数英总分的平均值。哪个班级最高？

In [15]:
df[df.选科.isin(["物理", "化学"])].groupby("班级").apply(lambda x: x.iloc[:, -4:-1].sum(1).mean()).idxmax()

2

### 3、
学生在该学期的总评计算方式是各次考试总分的加权平均值，其中周测成绩权重为50%（每次测验权重相等，即3.125%），期中权重为20%，期末权重为30%。请结合nlargest函数找出年级中总评前十的同学。

In [16]:
files = list(map(lambda i: "第%s次周测成绩"%(i+1), range(16)))
files += ["期中考试成绩", "期末考试成绩"]
path = "data/supplement/ex2/%s.csv"
dfs = list(map(lambda i: pd.read_csv(path%i), files))
df = pd.concat(dfs, keys=files).droplevel(1, axis=0)
df["总分"] = df.iloc[:, -4:].sum(1)
df = df.set_index("姓名", append=True)["总分"].unstack(0)
res = (df.values * np.r_[np.repeat(0.5/16, 16), [0.2, 0.3]]).sum(1)
students = pd.Series(res, index=df.index).nlargest(10)
students

姓名
黄秀荣    430.40000
包健     426.93125
左娟     424.38125
刘欢     423.68750
史阳     422.31250
李浩     421.68125
李琴     421.63125
齐峰     421.42500
郎佳     420.33125
戴杨     420.27500
dtype: float64

### 4、
请统计1班到8班文理科（物化生为理科，政史地为文科）期末考试总分前5的学生，结果格式如下，括号内的为选科分数

In [17]:
course_map = dict(zip(
    ["物理", "化学", "生物", "历史", "政治", "地理"],
    ["（理）"] * 3 + ["（文）"] * 3,
))
def func_top5_inner(df):
    df["总分"] = df.iloc[:, -5:-1].sum(1)
    df = df.sort_values("总分", ascending=False).iloc[:5]
    score = df["科目"].astype("string")
    res = df["姓名"] + "：" + df["选科"] + "（" + score + "）"
    return res.reset_index(drop=True)
def func_top5(df):
    df["文理"] = df.选科.replace(course_map)
    return df.groupby("文理").apply(func_top5_inner)
df = dfs[-1].query("班级 <= 8")
res = df.groupby("班级").apply(func_top5).T
res.columns = res.columns.map(lambda x: "%d班"%x[0] + x[1])
res

,1班（文）,1班（理）,2班（文）,2班（理）,3班（文）,3班（理）,4班（文）,4班（理）,5班（文）,5班（理）,6班（文）,6班（理）,7班（文）,7班（理）,8班（文）,8班（理）
0,秦利：地理（116）,张欢：物理（105）,燕丽娟：历史（103）,王淑英：化学（115）,黄萍：地理（96）,殷志强：物理（134）,何阳：地理（121）,田帆：物理（129）,李桂芝：历史（111）,林婷：物理（106）,齐峰：政治（93）,宋博：物理（114）,金阳：历史（108）,张畅：生物（123）,韩坤：地理（110）,王想：物理（115）
1,王雷：地理（116）,周颖：生物（111）,李飞：地理（104）,徐佳：化学（114）,郝丽娟：地理（105）,钟玉梅：物理（126）,卢飞：地理（108）,黄勇：物理（141）,罗建华：历史（94）,关斌：物理（139）,汤秀芳：地理（114）,杜海燕：物理（136）,陈丹丹：历史（89）,田磊：化学（108）,张林：历史（102）,刘娜：化学（102）
2,刘婷婷：地理（115）,陈云：化学（114）,戴宇：政治（104）,张勇：物理（125）,张飞：地理（111）,杜想：化学（106）,房娟：地理（117）,何伟：化学（118）,赵柳：历史（94）,彭桂兰：化学（101）,江莉：政治（105）,程桂珍：化学（97）,卢桂芝：历史（106）,吕雪：物理（127）,李小红：历史（105）,黎想：化学（118）
3,唐秀兰：历史（115）,陈柳：化学（103）,张建平：历史（105）,龙鑫：物理（121）,任玉梅：地理（107）,张洋：化学（116）,王帆：历史（110）,孙桂珍：物理（141）,谭鑫：地理（108）,叶平：物理（131）,张玉：地理（106）,窦畅：化学（125）,马亮：历史（111）,胡玉：化学（125）,冯磊：地理（99）,严琳：化学（125）
4,龙晨：地理（106）,王婷：化学（103）,李楠：历史（114）,周凤兰：生物（99）,王玉：地理（115）,蔡玲：化学（109）,王华：历史（100）,赵云：生物（92）,鲁瑞：历史（100）,钟辉：化学（119）,徐红：历史（91）,黎玉：物理（136）,杨桂兰：政治（100）,康华：化学（122）,陈楠：地理（113）,张桂香：物理（124）


### 5、
学生成绩的稳定性可以用每次考试在全年级相同选科学生中的总分排名标准差来度量，请计算每个班级的各科学生成绩稳定性的均值，结果格式如下：

In [ ]:
pd.DataFrame(
    np.random.rand(11, 6),
    index=pd.Index(range(1, 12), name="班级"),
    columns=pd.Index(
        ["物理", "化学", "生物", "历史", "地理", "政治"],
        name="选科",
    )
)

In [18]:
df = pd.concat(dfs, keys=files).droplevel(1, axis=0)
df = df.reset_index().rename(columns={"index": "考试"})
df["总分"] = df.iloc[:, -4:].sum(1)
df["排名"] = df.groupby(["选科", "考试"])["总分"].rank(method="min")
df = df.groupby(["班级", "姓名", "选科"])["排名"].std()
df = df.groupby(["班级", "选科"]).mean().unstack(-1)
res = df.reindex(["物理", "化学", "生物", "历史", "地理", "政治"], axis=1)
res

选科,物理,化学,生物,历史,地理,政治
班级,,,,,,
1,36.145785,71.123275,47.905771,37.686440,34.938192,26.934054
2,38.868956,70.116562,48.394368,39.495012,35.498685,26.682999
3,38.722614,75.090907,47.091807,38.271326,38.558896,27.574519
4,38.256357,70.934602,47.576877,38.507239,36.602089,28.351219
5,38.397798,72.132075,47.419773,36.919986,36.555557,24.545272
6,36.811151,68.205272,49.240808,38.288654,35.047769,28.235608
7,38.292788,71.099573,47.191711,37.796623,36.289223,27.433563
8,38.756076,71.971925,49.498229,36.839744,36.559538,27.615383
9,36.393109,65.635409,47.021174,39.539784,37.959470,27.108505


# Ex3：统计商品的审核情况
在data/supplement/ex3中存放了两个有关商品审核的信息表，“商品信息.csv”中记录了每个商品的ID号，唯一的识别码以及商品所属的类别，“申请与审核记录.csv”中记录了每个商品的审核信息。已知商品的审核流程如下：由申请人发起商品审核的申请，然后由审核人审核，审核的结果包括通过与不通过两种情况，若商品不通过审核则可以由另一位申请人再次发起申请，直到商品的审核通过。

In [399]:
df_info = pd.read_csv('data/supplement/ex3/商品信息.csv')

df_info.head()

,ID号,识别码,类别
0,ID 000001,CRtXJUK,T1
1,ID 000002,RGSxifC,Q1
2,ID 000003,AboduTp,S1
3,ID 000004,zlpUeMl,S2
4,ID 000005,IVQqhIK,S3


In [389]:
df_record = pd.read_csv('data/supplement/ex3/申请与审核记录.csv')

df_record.head()

,ID号,申请人,申请时间,审核人,审核时间,结果
0,ID 000001,\#+3((52\{,2020-04-19,~1=6\*183|,2020-05-03,未通过
1,ID 000001,8@75[1|2\*,2020-05-10,15![3\({59,2020-07-17,未通过
2,ID 000001,}!7)(#^0*7,2020-07-28,3`}04}%@75,2020-08-23,通过
3,ID 000002,|*{20#9|}5,2020-01-05,={`8]03*4+,2020-03-09,未通过
4,ID 000002,4~6%)455`[,2020-03-14,=$-36[)|8],2020-04-21,未通过


### 1、
有多少商品最终通过审核？


In [19]:
df_info = pd.read_csv('data/supplement/ex3/商品信息.csv')
df_record = pd.read_csv('data/supplement/ex3/申请与审核记录.csv')
(df_record["结果"] == "通过").sum()

10848

### 2、
各类别商品的通过率分别为多少？


In [20]:
df = df_record.merge(df_info, how="left", on="ID号")
df.groupby("类别").apply(lambda x: (x["结果"] == "通过").sum() / x["ID号"].nunique())

类别
Q1    0.395866
Q2    0.406604
S1    0.402618
S2    0.408443
S3    0.414896
T1    0.398679
T2    0.402781
dtype: float64

### 3、
对于类别为“T1”且最终状态为通过的商品，平均审核次数为多少？


In [21]:
grouper = df[df["类别"]=="T1"].groupby("ID号")["结果"]
grouper.count()[grouper.agg(lambda x: x.iloc[-1]=="通过")].mean()

5.547382372432074

### 4、
是否存在商品在上一次审核未完成时就提交了下一次审核申请？


In [22]:
idx = df["ID号"].duplicated().values
time1 = df["申请时间"].iloc[1:][idx[1:]].reset_index(drop=True)
time2 = df["审核时间"].iloc[:-1][idx[1:]].reset_index(drop=True)
(time1 >= time2).all() # True则不存在


True

### 5、
请对所有审核通过的商品统计第一位申请人和最后一位审核人的信息，返回格式如下：

In [ ]:
pd.DataFrame(
    {
        "ID号": ["ID 000001"]+["..."]*3,
        "类别":["T1"]+["..."]*3,
        "申请人":["\#+3((52\{"]+["..."]*3,
        "审核人":["3`}04}%@75"]+["..."]*3
    },
    index=[1,2,3,"..."]
)

###### 提示
groupby对象上也定义了head和tail方法。

In [23]:
res = df.groupby("ID号").apply(lambda x: pd.Series([x.iloc[0, 0], x.iloc[0, -1], x.iloc[0, 1], x.iloc[-1, 3]], index=["ID号", "类别", "申请人", "审核人"])).reset_index(drop=True)
res

,ID号,类别,申请人,审核人
0,ID 000001,T1,\#+3((52\{,3`}04}%@75
1,ID 000002,Q1,|*{20#9|}5,[3=5@|[0{9
2,ID 000003,S1,8!&|48\\##,5_*{@&#*9}
3,ID 000004,S2,8#=#)3_[10,^)1-\)#3*\
4,ID 000005,S3,=_@@9(2|#|,^7#10{&3^6
...,...,...,...,...
26827,ID 026828,S2,\##{~+^9!3,@`^=@!{0#@
26828,ID 026829,T1,!=1{)^%-@\,&]3]$)18=5
26829,ID 026830,T2,@`=(2@03^0,[_7\=*\(#8
26830,ID 026831,T2,64+%-|$$~#,_&}~(017*`


In [24]:
grouper = df.groupby("ID号")
df_head = grouper[["ID号", "类别", "申请人"]].head(1).reset_index(drop=True)
df_tail = grouper[["审核人"]].tail(1).reset_index(drop=True)
res2 = pd.concat([df_head, df_tail], axis=1)
res2.equals(res)

True